<a href="https://colab.research.google.com/github/lsmanoel/BasicOfPython/blob/master/BasicOfTensorflow/Tensorflow_howToUse_Dataset_Iterator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  Efficient approach of Datasets e Iterators with Tensorflow

A revision of [towardsdatascience.com/how-to-use-dataset-in-tensorflow](https://towardsdatascience.com/how-to-use-dataset-in-tensorflow-c758ef9e4428)

Oficial dataset guide: [www.tensorflow.org/guide/datasets](https://www.tensorflow.org/guide/datasets)

About Data Input Pipeline Performance: [tensorflow.org/guide/performance/datasets](https://www.tensorflow.org/guide/performance/datasets)

The **feed-dict** is a slowest possible way to pass information to Tensorflow. This way to feed the model in Tensorflow must be avoided. The fast way to feed data into the models is to use an input pipeline. The Tensorflow has a built-in API called Dataset to use fast pipelines to feed our models.

**Pipelining to performance of GPU and CPU integrated system:**

Pipelining overlaps the preprocessing and model execution of a training step. While the accelerator is performing training step N, the CPU is preparing the data for step N+1. Doing so reduces the step time to the maximum (as opposed to the sum) of the training and the time it takes to extract and transform the data.

Without pipelining, the CPU and the GPU/TPU sit idle much of the time:

![alt text](https://www.tensorflow.org/images/datasets_without_pipelining.png)

With pipelining, idle time diminishes significantly:

![alt text](https://www.tensorflow.org/images/datasets_with_pipelining.png)

The **tf.data** API provides a software pipelining mechanism through the **tf.data**.Dataset.prefetch transformation, which can be used to decouple the time data is produced from the time it is consumed. In particular, the transformation uses a background thread and an internal buffer to prefetch elements from the input dataset ahead of the time they are requested. Thus, to achieve the pipelining effect illustrated above, you can add prefetch(1) as the final transformation to your dataset pipeline (or prefetch(n) if a single training step consumes n elements).

Source:  [tensorflow.org/guide/performance/datasets](https://www.tensorflow.org/guide/performance/datasets)







In [0]:
dataset = dataset.batch(batch_size=FLAGS.batch_size)
dataset = dataset.prefetch(buffer_size=FLAGS.prefetch_buffer_size)

**Parallelize Data Transformation:**

When preparing a batch, input elements may need to be pre-processed. To this end, the tf.data API offers the **tf.data.Dataset.map** transformation, which applies a user-defined function (for example, parse_fn from the running example) to each element of the input dataset. Because input elements are independent of one another, **the pre-processing can be parallelized across multiple CPU cores**. To make this possible, the map transformation provides the **num_parallel_calls** argument to specify the level of parallelism. For example, the following diagram illustrates the effect of s**etting num_parallel_calls=2** to the map transformation:

![alt text](https://www.tensorflow.org/images/datasets_parallel_map.png)

Choosing the best value for the num_parallel_calls argument depends on your hardware, characteristics of your training data (such as its size and shape), the cost of your map function, and what other processing is happening on the CPU at the same time; a simple heuristic is to use the number of available CPU cores. For instance, if the machine executing the example above had 4 cores, it would have been more efficient to set num_parallel_calls=4. On the other hand, setting num_parallel_calls to a value much greater than the number of available CPUs can lead to inefficient scheduling, resulting in a slowdown.

Source:  [tensorflow.org/guide/performance/datasets](https://www.tensorflow.org/guide/performance/datasets)


In [0]:
dataset = dataset.map(map_func=parse_fn, num_parallel_calls=FLAGS.num_parallel_calls)

# Importing Data

To feed the model with a numpy array:

In [0]:
import numpy as np
import tensorflow as tf

# random vector of 20 elements with shape (32, 32, 4):
nElements = 20
input_x = np.random.sample((nElements, 32, 32, 4))

# create the dataset:
dataset = tf.data.Dataset.from_tensor_slices(input_x)

The data divided into features and labels:

In [0]:
import numpy as np
import tensorflow as tf

nElements = 20
x_data = np.random.sample((nElements, 32, 32, 4)) #features
y_data = np.random.sample((nElements, 1)) #label

dataset = tf.data.Dataset.from_tensor_slices((x_data,y_data))

Use placeholders to dynamically change the data inside the Dataset:

In [0]:
import numpy as np
import tensorflow as tf

x_data = tf.placeholder(tf.float32, shape=[None, 32, 32, 4])
dataset = tf.data.Dataset.from_tensor_slices(x_data)

To feed from generators:

In [5]:
import numpy as np
import tensorflow as tf

# from generator
sequence = np.array([[[1],[2],[3]],
                     [[4],[5],[6]],
                     [[7],[8],[9]]])

# sequence = np.array([[1],[3],[2]])
# sequence = np.arange(4)
def generator():
    for row in sequence:
      for x in row:
        yield x*x

dataset = tf.data.Dataset().batch(1).from_generator(generator,
                                                    output_types= tf.int64, 
                                                    output_shapes=(tf.TensorShape(None)))

iter = dataset.make_initializable_iterator()
x = iter.get_next()

with tf.Session() as sess:
    sess.run(iter.initializer)
    for i in range(sequence.shape[0]*sequence.shape[1]*sequence.shape[2]):
      print("iter:{} => {}".format(i+1, sess.run(x)))


iter:1 => [1]
iter:2 => [4]
iter:3 => [9]
iter:4 => [16]
iter:5 => [25]
iter:6 => [36]
iter:7 => [49]
iter:8 => [64]
iter:9 => [81]


In [6]:
sequence.shape

(3, 3, 1)

To read a csv file into a dataset:

In [7]:
from google.colab import drive 
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [8]:
import numpy as np
import tensorflow as tf

# load a csv
CSV_PATH = 'gdrive/My Drive/Colab Notebooks/AprendizagemNaoSupervisionada/self-organizingMaps/datasets/wines.csv'
dataset = tf.data.experimental.make_csv_dataset(CSV_PATH, batch_size=32)
iter = dataset.make_one_shot_iterator()
next = iter.get_next()
print(next) # next is a dict with key=columns names and value=column data
inputs, labels = next['Class'], next['Alcohol']

with  tf.Session() as sess:
    sess.run([inputs, labels])

OrderedDict([('Class', <tf.Tensor 'IteratorGetNext_1:3' shape=(32,) dtype=int32>), ('Alcohol', <tf.Tensor 'IteratorGetNext_1:2' shape=(32,) dtype=float32>), ('Malic acid', <tf.Tensor 'IteratorGetNext_1:8' shape=(32,) dtype=float32>), (' Ash', <tf.Tensor 'IteratorGetNext_1:1' shape=(32,) dtype=float32>), (' Alcalinity of ash', <tf.Tensor 'IteratorGetNext_1:0' shape=(32,) dtype=float32>), ('Magnesium', <tf.Tensor 'IteratorGetNext_1:7' shape=(32,) dtype=int32>), ('Total phenols', <tf.Tensor 'IteratorGetNext_1:13' shape=(32,) dtype=float32>), ('Flavanoids', <tf.Tensor 'IteratorGetNext_1:5' shape=(32,) dtype=float32>), ('Nonflavanoid phenols', <tf.Tensor 'IteratorGetNext_1:9' shape=(32,) dtype=float32>), ('Proanthocyanins', <tf.Tensor 'IteratorGetNext_1:11' shape=(32,) dtype=float32>), ('Color intensity', <tf.Tensor 'IteratorGetNext_1:4' shape=(32,) dtype=float32>), ('Hue', <tf.Tensor 'IteratorGetNext_1:6' shape=(32,) dtype=float32>), ('OD280/OD315 of diluted wines', <tf.Tensor 'IteratorGet

# Iterator

The iterator will give us the ability to iterate through the dataset and retrieve the real values of the data.

There exist four types of iterators:



*   **One Shot:** It can iterate once through a dataset, you cannot feed any value to it.
*   **Initializable:** You can dynamically change calling its initializer operation and passing the new data with feed_dict . It’s basically a bucket that you can fill with stuff.
*   **Reinitializable: ** It can be initialised from different Dataset. Very useful when you have a training dataset that needs some additional transformation, eg. shuffle, and a testing dataset. It’s like using a tower crane to select a different container.
*   **Feedable:** It can be used to select with iterator to use. Following the previous example, it’s like a tower crane that selects which tower crane to use to select which container to take. In my opinion is useless.



**One Shot:**

In [9]:
import numpy as np
import tensorflow as tf

input_array = np.arange(3)
# make a dataset from a numpy array
dataset = tf.data.Dataset.from_tensor_slices(input_array)

# create the iterator
iter = dataset.make_one_shot_iterator()
x = iter.get_next()

with tf.Session() as sess:
    print(sess.run(x), sess.run(x), sess.run(x))

0 1 2


**Initializable Iterator:**

Create a dataset with a placeholder to build a dynamic dataset in which we can change the data source at runtime. The placeholder need be initialize by feed-dict mechanism. This time we call **make_initializable_iterator** method from [Dataset](https://www.tensorflow.org/api_docs/python/tf/data/Dataset).

Then, inside the sess scope, we run the initializer operation in order to pass our data, in this case a random numpy array:

In [10]:
import numpy as np
import tensorflow as tf

# using a placeholder
x = tf.placeholder(tf.float32, shape=[None, 32, 32, 4])
dataset = tf.data.Dataset.from_tensor_slices(x)

nElements = 20
input_x = np.random.sample((nElements, 32, 32, 4))

iter = dataset.make_initializable_iterator() # create the iterator
y = iter.get_next()

with tf.Session() as sess:
    # feed the placeholder with data
    sess.run(iter.initializer, feed_dict={x: input_x}) 
    print(sess.run(y).shape)
    print(sess.run(y).shape)
    print(sess.run(y).shape)

(32, 32, 4)
(32, 32, 4)
(32, 32, 4)


A real common scenario (train set and a test set):

In [11]:
import numpy as np
import tensorflow as tf

train_data = (np.random.sample((100,2)), np.random.sample((100,1)))
test_data = (np.array([[1,2]]), np.array([[0]]))

x, y = tf.placeholder(tf.float32, shape=[None,2]), tf.placeholder(tf.float32, shape=[None,1])
dataset = tf.data.Dataset.from_tensor_slices((x, y))

train_data = (np.random.sample((100,2)), np.random.sample((100,1)))
test_data = (np.array([[1,2]]), np.array([[0]]))

# initializable iterator to switch between dataset
iter = dataset.make_initializable_iterator()
features, labels = iter.get_next()

EPOCHS = 10
with tf.Session() as sess:
#     initialise iterator with train data
    sess.run(iter.initializer, feed_dict={ x: train_data[0], y: train_data[1]})
  
    for _ in range(EPOCHS):
        sess.run([features, labels])
        
#     switch to test data
    sess.run(iter.initializer, feed_dict={ x: test_data[0], y: test_data[1]})
    print(sess.run([features, labels]))

[array([1., 2.], dtype=float32), array([0.], dtype=float32)]


**Reinitializable Iterator:**

The concept is similar to before, we want to dynamic switch between data. But instead of feed new data to the same dataset, we switch dataset. As before, we want to have a train dataset and a test dataset

In [12]:
import numpy as np
import tensorflow as tf

# making fake data using numpy
train_data = (np.random.sample((100,2)), np.random.sample((100,1)))
test_data = (np.random.sample((10,2)), np.random.sample((10,1)))

# create two datasets, one for training and one for test
train_dataset = tf.data.Dataset.from_tensor_slices(train_data)
test_dataset = tf.data.Dataset.from_tensor_slices(test_data)

# create a iterator of the correct shape and type
iter = tf.data.Iterator.from_structure(train_dataset.output_types,
                                           train_dataset.output_shapes)

# create the initialisation operations
train_init_op = iter.make_initializer(train_dataset)
test_init_op = iter.make_initializer(test_dataset)

# get the next element as before
features, labels = iter.get_next()

# Reinitializable iterator to switch between Datasets
EPOCHS = 10
with tf.Session() as sess:
    sess.run(train_init_op) # switch to train dataset
    
    for _ in range(EPOCHS):
        sess.run([features, labels])
        
    sess.run(test_init_op) # switch to val dataset
    print(sess.run([features, labels]))

[array([0.77451975, 0.54779441]), array([0.65103328])]


**Feedable Iterator:**

This is very similar to the **Reinitializable Iterator**, but instead of switch between datasets, it switch between iterators.
[tensorflow.org/programmers_guide/datasets#creating_an_iterator](https://www.tensorflow.org/programmers_guide/datasets#creating_an_iterator)

In [13]:
import numpy as np
import tensorflow as tf

# making fake data using numpy
train_data = (np.random.sample((100,2)), np.random.sample((100,1)))
test_data = (np.random.sample((10,2)), np.random.sample((10,1)))

# create placeholder
x, y = tf.placeholder(tf.float32, shape=[None,2]), tf.placeholder(tf.float32, shape=[None,1])

# create two datasets, one for training and one for test
train_dataset = tf.data.Dataset.from_tensor_slices((x,y))
test_dataset = tf.data.Dataset.from_tensor_slices((x,y))

# create the iterators from the dataset
train_iterator = train_dataset.make_initializable_iterator()
test_iterator = test_dataset.make_initializable_iterator()

# same as in the doc: 
# https://www.tensorflow.org/programmers_guide/datasets#creating_an_iterator
handle = tf.placeholder(tf.string, shape=[])
iter = tf.data.Iterator.from_string_handle(
    handle, train_dataset.output_types, train_dataset.output_shapes)
next_elements = iter.get_next()

# feedable iterator to switch between iterators
EPOCHS = 10
with tf.Session() as sess:
    train_handle = sess.run(train_iterator.string_handle())
    test_handle = sess.run(test_iterator.string_handle())
    
    # initialise iterators. 
    sess.run(train_iterator.initializer, feed_dict={ x: train_data[0], y: train_data[1]})
    sess.run(test_iterator.initializer, feed_dict={ x: test_data[0], y: test_data[1]})
    
    for _ in range(EPOCHS):
        x,y = sess.run(next_elements, feed_dict = {handle: train_handle})
        print(x, y)
        
    x,y = sess.run(next_elements, feed_dict = {handle: test_handle})
    print(x,y)

[0.40486744 0.09454836] [0.45837525]
[0.40568185 0.25635305] [0.9361035]
[0.18568978 0.8688876 ] [0.88782215]
[0.03910578 0.6690394 ] [0.6457239]
[0.30962598 0.42874312] [0.72969776]
[0.46682042 0.13815157] [0.23211609]
[0.22990501 0.68973386] [0.10259616]
[0.39459708 0.09110982] [0.9414525]
[0.6679728 0.4056337] [0.74841356]
[0.7971969  0.94630957] [0.40634608]
[0.59526926 0.2725911 ] [0.70964456]


**Consuming data:**

In order to pass the data to a model we have to just pass the tensors generated from **get_next()**

In [14]:
import numpy as np
import tensorflow as tf

# using two numpy arrays
features, labels = (np.array([np.random.sample((100,2))]), 
                    np.array([np.random.sample((100,1))]))

BATCH_SIZE = 16
dataset = tf.data.Dataset.from_tensor_slices((features,labels)).repeat().batch(BATCH_SIZE)

iter = dataset.make_one_shot_iterator()
x, y = iter.get_next()

# make a simple model
net = tf.layers.dense(x, 8, activation=tf.tanh) # pass the first value from iter.get_next() as input
net = tf.layers.dense(net, 8, activation=tf.tanh)
prediction = tf.layers.dense(net, 1, activation=tf.tanh)

loss = tf.losses.mean_squared_error(prediction, y) # pass the second value from iter.get_net() as label
train_op = tf.train.AdamOptimizer().minimize(loss)

EPOCHS = 10
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(EPOCHS):
        _, loss_value = sess.run([train_op, loss])
        print("Iter: {}, Loss: {:.4f}".format(i, loss_value))

Iter: 0, Loss: 0.5263
Iter: 1, Loss: 0.5098
Iter: 2, Loss: 0.4936
Iter: 3, Loss: 0.4778
Iter: 4, Loss: 0.4623
Iter: 5, Loss: 0.4471
Iter: 6, Loss: 0.4323
Iter: 7, Loss: 0.4179
Iter: 8, Loss: 0.4038
Iter: 9, Loss: 0.3901


# Useful Stuff

**Batch:**

The Dataset API has the method **batch(BATCH_SIZE)** that automatically batches the dataset with the provided size.


In [15]:
import numpy as np
import tensorflow as tf

# BATCHING
BATCH_SIZE = 4
x = np.random.sample((100,2))
# make a dataset from a numpy array
dataset = tf.data.Dataset.from_tensor_slices(x).batch(BATCH_SIZE)

iter = dataset.make_one_shot_iterator()
y = iter.get_next()

with tf.Session() as sess:
    print(sess.run(y)) 

[[0.19498597 0.07937528]
 [0.34385483 0.55348209]
 [0.7791454  0.13523891]
 [0.21017553 0.11101988]]


**Repeat:**

Using **.repeat()** we can specify the number of times we want the dataset to be iterated. If no parameter is passed it will loop forever, usually is good to just loop forever and directly control the number of epochs with a standard loop.


In [78]:
import numpy as np
import tensorflow as tf

#Random Sequence Generator

# tf.data.Dataset.range(9) -> Creates a Dataset of a step-separated range of values.

# .shuffle(buffer_size=10) -> shuffle 10 elements of dataset
# shuffle(
#     buffer_size,
#     seed=None,
#     reshuffle_each_iteration=None -> Default = True
# )


# .repeat(None) or .repeat(-1) -> Repeat indefinitely 

n = tf.data.Dataset.range(9).shuffle(buffer_size=10).repeat(None).make_one_shot_iterator().get_next()
with tf.Session() as sess:
  [print(sess.run(n)) for _ in range(10)]

7
8
6
0
5
2
3
1
4
8


**Shuffle:**

We can shuffle the Dataset by using the method **shuffle() ** that shuffles the dataset by default every epoch.

We can also set the parameter **buffer_size** , a fixed size buffer from which the next element will be uniformly chosen from.

In [108]:
import numpy as np
import tensorflow as tf

# BATCHING
BATCH_SIZE = 2 # 6 elements and 3 BATCHS of size 2
x = np.array([[0],[1],[2],[3],[4],[5]])

# make a dataset from a numpy array
dataset = tf.data.Dataset.from_tensor_slices(x)
dataset = dataset.shuffle(buffer_size=2)
dataset = dataset.batch(BATCH_SIZE)

iter = dataset.make_one_shot_iterator()
el = iter.get_next()

with tf.Session() as sess:
    print("Without repeat:")
    print("--------------")
    print("First Batch:\n",  sess.run(el))
    print("Second Batch:\n", sess.run(el))
    print("Third Batch:\n",  sess.run(el))
    print("\n")
    
# WITH REPEAT:
# make a dataset from a numpy array
dataset = tf.data.Dataset.from_tensor_slices(x)
dataset = dataset.shuffle(buffer_size=2)
dataset = dataset.batch(BATCH_SIZE)
dataset = dataset.repeat(None)

iter = dataset.make_one_shot_iterator()
el = iter.get_next()

with tf.Session() as sess:
    print("With repeat:")
    print("--------------")
    print("First Batch:\n",  sess.run(el))
    print("Second Batch:\n", sess.run(el))
    print("Third Batch:\n",  sess.run(el))
    print("...")
    print("First Batch:\n",  sess.run(el))
    print("Second Batch:\n", sess.run(el))
    print("Third Batch:\n",  sess.run(el))
    print("\n")

Without repeat:
--------------
First Batch:
 [[0]
 [2]]
Second Batch:
 [[1]
 [4]]
Third Batch:
 [[3]
 [5]]


With repeat:
--------------
First Batch:
 [[0]
 [2]]
Second Batch:
 [[3]
 [1]]
Third Batch:
 [[5]
 [4]]
...
First Batch:
 [[1]
 [0]]
Second Batch:
 [[2]
 [4]]
Third Batch:
 [[3]
 [5]]




# Map

You can apply a custom function to each member of a dataset using the **map** method. In the following example we multiply each element by two:

In [110]:
import numpy as np
import tensorflow as tf

x = np.array([[1],[2],[3],[4]])
# make a dataset from a numpy array
dataset = tf.data.Dataset.from_tensor_slices(x)

# MAP:
dataset = dataset.map(lambda x: x*2)

iter = dataset.make_one_shot_iterator()
el = iter.get_next()

with tf.Session() as sess:
#     this will run forever
        for _ in range(len(x)):
            print(sess.run(el))

[2]
[4]
[6]
[8]


# Full example

**Initializable iterator:**

In [113]:
import numpy as np
import tensorflow as tf

# Wrapping all together -> Switch between train and test set using Initializable iterator
EPOCHS = 10

# create a placeholder to dynamically switch between batch sizes
BATCH_SIZE = 4
batch_size = tf.placeholder(tf.int64)

x, y = tf.placeholder(tf.float32, shape=[None,2]), tf.placeholder(tf.float32, shape=[None,1])
dataset = tf.data.Dataset.from_tensor_slices((x, y)).batch(batch_size).repeat()

# using two numpy arrays
train_data = (np.random.sample((100,2)), np.random.sample((100,1)))
test_data = (np.random.sample((20,2)), np.random.sample((20,1)))

iter = dataset.make_initializable_iterator()
features, labels = iter.get_next()

# make a simple model
net = tf.layers.dense(features, 8, activation=tf.tanh) # pass the first value from iter.get_next() as input
net = tf.layers.dense(net, 8, activation=tf.tanh)
prediction = tf.layers.dense(net, 1, activation=tf.tanh)

loss = tf.losses.mean_squared_error(prediction, labels) # pass the second value from iter.get_net() as label
train_op = tf.train.AdamOptimizer().minimize(loss)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    # initialise iterator with train data
    sess.run(iter.initializer, feed_dict={ x: train_data[0], y: train_data[1], batch_size: BATCH_SIZE})
    
    print('Training...')
    for i in range(EPOCHS):
        tot_loss = 0
        for _ in range(BATCH_SIZE):
            _, loss_value = sess.run([train_op, loss])
            tot_loss += loss_value
        print("Iter: {}, Loss: {:.4f}".format(i, tot_loss / BATCH_SIZE))
    # initialise iterator with test data
    sess.run(iter.initializer, feed_dict={ x: test_data[0], y: test_data[1], batch_size: test_data[0].shape[0]})
    print('Test Loss: {:4f}'.format(sess.run(loss)))

Training...
Iter: 0, Loss: 0.1061
Iter: 1, Loss: 0.1186
Iter: 2, Loss: 0.0487
Iter: 3, Loss: 0.1480
Iter: 4, Loss: 0.1530
Iter: 5, Loss: 0.1027
Iter: 6, Loss: 0.0839
Iter: 7, Loss: 0.1120
Iter: 8, Loss: 0.0325
Iter: 9, Loss: 0.0894
Test Loss: 0.108598


**Reinitializable Iterator:**

In [114]:
import numpy as np
import tensorflow as tf

# Wrapping all together -> Switch between train and test set using Reinitializable iterator
EPOCHS = 10

# create a placeholder to dynamically switch between batch sizes
BATCH_SIZE = 4
batch_size = tf.placeholder(tf.int64)

x, y = tf.placeholder(tf.float32, shape=[None,2]), tf.placeholder(tf.float32, shape=[None,1])
train_dataset = tf.data.Dataset.from_tensor_slices((x,y)).batch(batch_size).repeat()
test_dataset = tf.data.Dataset.from_tensor_slices((x,y)).batch(batch_size) # always batch even if you want to one shot it

# using two numpy arrays
train_data = (np.random.sample((100,2)), np.random.sample((100,1)))
test_data = (np.random.sample((20,2)), np.random.sample((20,1)))

# create a iterator of the correct shape and type
iter = tf.data.Iterator.from_structure(train_dataset.output_types,
                                           train_dataset.output_shapes)
features, labels = iter.get_next()
# create the initialisation operations
train_init_op = iter.make_initializer(train_dataset)
test_init_op = iter.make_initializer(test_dataset)

# make a simple model
net = tf.layers.dense(features, 8, activation=tf.tanh) # pass the first value from iter.get_next() as input
net = tf.layers.dense(net, 8, activation=tf.tanh)
prediction = tf.layers.dense(net, 1, activation=tf.tanh)

loss = tf.losses.mean_squared_error(prediction, labels) # pass the second value from iter.get_net() as label
train_op = tf.train.AdamOptimizer().minimize(loss)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    # initialise iterator with train data
    sess.run(train_init_op, feed_dict = {x : train_data[0], y: train_data[1], batch_size: 16})
    
    print('Training...')
    for i in range(EPOCHS):
        tot_loss = 0
        for _ in range(BATCH_SIZE):
            _, loss_value = sess.run([train_op, loss])
            tot_loss += loss_value
            
        print("Iter: {}, Loss: {:.4f}".format(i, tot_loss / BATCH_SIZE))
    
    # initialise iterator with test data
    sess.run(test_init_op, feed_dict = {x : test_data[0], y: test_data[1], batch_size:len(test_data[0])})
    print('Test Loss: {:4f}'.format(sess.run(loss)))

Training...
Iter: 0, Loss: 0.3457
Iter: 1, Loss: 0.2616
Iter: 2, Loss: 0.2503
Iter: 3, Loss: 0.2147
Iter: 4, Loss: 0.1785
Iter: 5, Loss: 0.1448
Iter: 6, Loss: 0.1349
Iter: 7, Loss: 0.1093
Iter: 8, Loss: 0.0967
Iter: 9, Loss: 0.0935
Test Loss: 0.085259
